Before running the file Upload all your data set on your goole drive in a zip format

In [ ]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install face_recognition torch torchvision
!mkdir '/content/drive/My Drive/FF_REAL_Face_only_data'

In [ ]:
#before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set harware accelarotor as GPU)
#download and unzip the data from google drive Colab environment
from google_drive_downloader import GoogleDriveDownloader as gdd
#use only file id of the link
#Note: Below link is just an example, Not an actual link. Actual Links are in ReadMe file
#https://drive.google.com/file/d/1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07/view?usp=sharing
url = '1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07'
gdd.download_file_from_google_drive(file_id=url, dest_path='./data.zip', unzip=True)

In [ ]:
# Get the average frame count 
import json
import glob
import numpy as np
import cv2
import copy

# Change the path accordingly
video_files = glob.glob('/content/Real videos/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1

frame_count = []
for video_file in video_files:
	cap = cv2.VideoCapture(video_file)
	if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150):
		video_files.remove(video_file)
		continue
	frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
  
print("Frames: ", frame_count)
print("Total number of videos: ", len(frame_count))
print("Average frame per video:", np.mean(frame_count))

In [ ]:
# Extract frame
def frame_extract(path):
	vidObj = cv2.VideoCapture(path) 
	success = 1
	while success:
		success, image = vidObj.read()
		if success: yield image

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm


# process the frames
def create_face_videos(path_list,out_dir):
    already_present_count = glob.glob(out_dir + '*.mp4')
    print("No of videos already present ", len(already_present_count))
    
    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, path.split('/')[-1])
        file_exists = glob.glob(out_path)
        if(len(file_exists) != 0):
            print("File Already exists: ", out_path)
            continue
        
        frames = []
        out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (112, 112))
        
        for idx,frame in enumerate(frame_extract(path)):
            if idx <= 150:
                frames.append(frame)
                if len(frames) == 4:
                    faces = face_recognition.batch_face_locations(frames)
                    for i,face in enumerate(faces):
                        if len(face) != 0:top, right, bottom, left = face[0]
                            
                        try: out.write(cv2.resize(frames[i][top:bottom, left:right, :], (112,112)))
                        except: pass
                    frames = []
                    
        try: del top, right, bottom, left
        except: pass
        out.release()

In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/FF_REAL_Face_only_data/')